# **Problem 5: Creating a Deepfake Video**

## Get Image and Video Content

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

#target-face and motion model can be found in the deepfake folder and must be uploaded into the notebook
source_image = imageio.imread('target-face.png')
source_image2 = imageio.imread('target-face2.png')
reader = imageio.get_reader('motion-model.mp4')
reader2 = imageio.get_reader('motion-model2.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
source_image2 = resize(source_image2, (256, 256))[..., :3]

def resize_vid(reader):
  fps = reader.get_meta_data()['fps']
  driving_video = []
  try:
      for im in reader:
          driving_video.append(im)
  except RuntimeError:
      pass
  reader.close()
  return [resize(frame, (256, 256))[..., :3] for frame in driving_video]

driving_video = resize_vid(reader)
driving_video2 = resize_vid(reader2)

#display the image and video

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

## Setup Model


In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256
Receiving objects: 100% (256/256), 72.13 MiB | 36.95 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [ ]:
cd first-order-model

/content/first-order-model


In [ ]:
#get drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#get the checkpoints
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

## Run Deepfake Animations

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

#relative keyboard displacement
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 373/373 [00:17<00:00, 20.73it/s]


Interesting Note: the deepfake clip produces a few blinks where the original clip has no blinks at all

In [ ]:
#absolute coordinates
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 373/373 [00:18<00:00, 20.71it/s]


Second Example

In [ ]:
#Pre-Prediction
HTML(display(source_image2, driving_video2).to_html5_video())

In [ ]:
#relative keyboard displacement
predictions = make_animation(source_image2, driving_video2, generator, kp_detector, relative=True)
HTML(display(source_image2, driving_video2, predictions).to_html5_video())

100%|██████████| 313/313 [00:16<00:00, 19.27it/s]


In [ ]:
#absolute coordinates
predictions = make_animation(source_image2, driving_video2, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image2, driving_video2, predictions).to_html5_video())

100%|██████████| 313/313 [00:16<00:00, 19.32it/s]
